In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = llm | parser

In [4]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {
    "configurable": {
        "session_id": "chat"
    }
}

In [5]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="사용자의 질문에 2문장 이내로 짧게 대답해"),
    HumanMessage(content="오늘은 피자를 먹어야지!"),
    AIMessage(content="정말 좋은 생각이야. 음료는 무엇으로 할 거야?"),
    HumanMessage(content="내일은 수영을 가야지!"),
    AIMessage(content="수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?"),
    HumanMessage(content="주말에는 영화를 보러 갈 거야!"),
    AIMessage(content="주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?")
]

In [6]:
with_message_history.invoke(
    messages + [HumanMessage(content="주말에 내가 뭐하러 간다고 했지?")],
    config=config
)

'주말에 영화를 보러 간다고 했어! 계획이 잘 되길 바라.'

In [7]:
with_message_history.invoke(
    [HumanMessage(content="방금 내가 무슨 질문을 했지?")],
    config=config
)

'주말에 뭐하러 간다고 했는지 물어보셨어요. 답변은 주말에 영화를 보러 간다고 하셨죠.'

In [10]:
with_message_history.invoke(
    [HumanMessage(content="내가 수영은 언제 간다고 했지?")],
    config=config
)

[chain/start] [chain:RunnableWithMessageHistory] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableWithMessageHistory > chain:load_history] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableWithMessageHistory > chain:load_history] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:RunnableWithMessageHistory > chain:check_sync_or_async] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableWithMessageHistory > chain:check_sync_or_async > chain:RunnableSequence] Entering Chain run with input:
[inputs]
[llm/start] [chain:RunnableWithMessageHistory > chain:check_sync_or_async > chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: 사용자의 질문에 2문장 이내로 짧게 대답해\nHuman: 오늘은 피자를 먹어야지!\nAI: 정말 좋은 생각이야. 음료는 무엇으로 할 거야?\nHuman: 내일은 수영을 가야지!\nAI: 수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?\nHuman: 주말에는 영화를 보러 갈 거야!\nAI: 주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?\nHuman: 주말에 내가 뭐하러 간다고 했지?\nAI: 주말에 영화를 보러 간다고 했어! 계

'내일 수영을 간다고 하셨습니다. 재미있게 다녀오세요!'

In [11]:
import langchain
langchain.debug = False

In [12]:
config_01 = {
    "configurable": {
        "session_id": "chat_01"
    }
}

with_message_history.invoke(
    [HumanMessage(content="내일은 수영을 가야지!")],
    config=config_01
)

'좋은 생각이에요! 수영은 건강에 매우 좋고 스트레스 해소에도 도움이 됩니다. 어떤 계획을 세우셨나요? 수영을 얼마나 자주 하시나요?'

In [13]:
with_message_history.invoke(
    [HumanMessage(content="내일 뭐하러 간다고 했지?")],
    config=config_01
)

'내일 수영을 가기로 하셨다고 하셨죠! 수영을 통해 즐거운 시간 보내시길 바랍니다. 혹시 다른 계획이나 준비할 것이 필요하신가요?'

In [14]:
config_02 = {
    "configurable": {
        "session_id": "chat_02"
    }
}

with_message_history.invoke(
    [HumanMessage(content="언제 간다고 했지?")],
    config=config_02
)

'어떤 상황에 대해 말씀하시는 건가요? 구체적인 정보나 맥락을 알려주시면 더 정확하게 답변해드릴 수 있을 것 같습니다.'

In [15]:
with_message_history.invoke(
    [HumanMessage(content="언제 간다고 했지?")],
    config=config_01
)

'정확한 시간은 언급하지 않으셨습니다. 내일 수영을 간다고 하셨는데, 특정 시간이나 장소에 대한 계획이 있으신가요? 일정이 어떻게 되는지 말씀해 주시면 도움이 될 것 같아요!'